In [3]:
import open3d as o3d
import numpy as np
import math
import copy

In [4]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [5]:
def sample_mesh_to_cloud(mesh, sample_number=100000, color = [1, 0.706, 0]):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color(color)
    return pcd

In [44]:
cloud = o3d.geometry.PointCloud()

In [45]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.908, -0.740, 1.378, 1, 0, 0, 0]).A)
cloud += box_cloud

In [46]:
box1 = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud1 = sample_mesh_to_cloud(box1, 5000, [0.5,0.5,0.5])
box_cloud1.transform(pose2matrix([2.905, -0.732, 1.119, 1, 0, 0, 0]).A)
cloud += box_cloud1

In [47]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.904, -0.361, 0.872, 1, 0, 0, 0]).A)
cloud += box_cloud

In [48]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.904, -0.737, 0.869, 1, 0, 0, 0]).A)
cloud += box_cloud

In [49]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.913, -0.360, 0.615, 1, 0, 0, 0]).A)
cloud += box_cloud

In [50]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.913, -0.739, 0.606, 1, 0, 0, 0]).A)
cloud += box_cloud

In [51]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.907, -1.109, 0.606, 1, 0, 0, 0]).A)
cloud += box_cloud

In [52]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.915, -0.360, 0.358, 1, 0, 0, 0]).A)
cloud += box_cloud

In [53]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.914, -0.739, 0.355, 1, 0, 0, 0]).A)
cloud += box_cloud

In [54]:
box = o3d.geometry.TriangleMesh.create_box(width=0.47, height=0.36, depth=0.26)
box_cloud = sample_mesh_to_cloud(box, 5000)
box_cloud.transform(pose2matrix([2.907, -1.109, 0.356, 1, 0, 0, 0]).A)
cloud += box_cloud

In [55]:
o3d.visualization.draw_geometries([cloud])

In [58]:
target = cloud

In [59]:
add_sign = True
if add_sign:
    sign_on_front = o3d.geometry.TriangleMesh.create_box(width=0.01, height=0.01, depth=0.01)
    sign_cloud = sample_mesh_to_cloud(sign_on_front, 1000)
    sign_cloud.transform(pose2matrix([1.75 + 1.2, 0, 0.07 + 0.26, 1, 0, 0, 0]).A)
    target =  target + sign_cloud
o3d.visualization.draw_geometries([target])

In [20]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud

def get_picked_indices(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name="Resgistration: Pick Corresponding Points")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries(
        [source_temp, target], window_name="Registration Visualization"
    )
    return source_temp + target

In [21]:
source = load_point_cloud("/home/bot/dev/loading_projects/onsite_data/0115_calibration/2023-01-15_193341_870368126_+0800_cycle_0_percipio_tof_right.pcd")
source_ids = get_picked_indices(source)
print(len(source_ids))
print(source_ids)

11
[18846, 19812, 40421, 40864, 41687, 42122, 42167, 43343, 69414, 69115, 68812]


In [60]:
target_ids = get_picked_indices(target)
print(len(target_ids))
print(target_ids)

11
[31252, 46024, 1273, 1674, 18570, 25855, 26750, 40816, 11751, 38898, 36067]


In [61]:
corr = np.zeros((len(source_ids), 2))
corr[:, 0] = source_ids
corr[:, 1] = target_ids

p2p = o3d.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(
    source, target, o3d.utility.Vector2iVector(corr)
)
print(trans_init)
draw_registration_result(source, target, trans_init)

[[-0.26439923 -0.13069072  0.95551713  0.82608441]
 [-0.043718    0.99138124  0.1234989  -0.91060708]
 [-0.96342191 -0.00912028 -0.26783397  2.03411621]
 [ 0.          0.          0.          1.        ]]


geometry::PointCloud with 150003 points.

In [62]:
reg_p2p = o3d.registration.registration_icp(
    source,
    target,
    0.005,
    trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
)

transform = reg_p2p.transformation
print("Transformation Matrix: \n{}\n".format(transform))

draw_registration_result(source, target, transform)

pose = matrix2pose(transform)
print(pose)

Transformation Matrix: 
[[-0.26919365 -0.10579285  0.95725788  0.8227459 ]
 [-0.03426401  0.99437121  0.10025899 -0.86084309]
 [-0.96247637 -0.00581041 -0.27130331  2.04105511]
 [ 0.          0.          0.          1.        ]]

[ 0.8227459  -0.86084309  2.04105511  0.60288354 -0.0439842   0.79606347
  0.02966113]
